# Fill-in EITC and CTC

Matt Bruenig of People's Policy Project proposed a "fill-in" tax credit to effectively remove the work requirements from the Earned Income Tax Credit and Child Tax Credit.
Bruenig proposed it as a state-level reform, but we have implemented it as a federal reform which can be broken down by any state, since it does not depend on any other state-level policies.

In [4]:
from openfisca_us import IndividualSim
from openfisca_us.model_api import *
from openfisca_core import periods

import pandas as pd
import plotly.express as px


def modify_parameters(parameters):
    parameters.contrib.peoples_policy_project.fill_in_eitc_ctc.update(
        period=periods.period("year:2022:1"), value=True
    )
    return parameters


class enact_fill_in(Reform):
    def apply(self):
        self.modify_parameters(modify_parameters)


LIGHT_GRAY = "#F5F5F5"
GRAY = "#BDBDBD"
BLUE = "#5091cc"
LIGHT_BLUE = "lightblue"
DARK_BLUE = "darkblue"

COLOR_MAP = {"0": GRAY, "1": LIGHT_BLUE, "2": BLUE, "3": DARK_BLUE}


def make(adults, children):
    sim = IndividualSim(enact_fill_in, year=2022)
    sim.add_person(name="head", age=25)
    members = ["head"]
    if adults == 2:
        sim.add_person(name="spouse")
        members += ["spouse"]
    for i in range(children):
        child = "child{}".format(i)
        sim.add_person(name=child, age=6)
        members += [child]
    sim.add_tax_unit(name="tax_unit", members=members)
    sim.add_household(name="household", members=members)
    sim.vary("employment_income", max=60_000, step=100)
    return pd.DataFrame(
        dict(
            employment_income=sim.calc("employment_income")[0],
            fill_in_eitc_ctc=sim.calc("fill_in_eitc_ctc")[0].round(),
            mtr=-sim.deriv("fill_in_eitc_ctc", "employment_income", wrt_target="head"),
            adults=adults,
            children=str(children),
        )
    )


# Make a table of EITC amounts for different numbers of adults and children.
l = []
for adults in range(1, 3):
    for children in range(0, 4):
        l.append(make(adults, children))

df = pd.concat(l)

LABELS = dict(
    employment_income="Employment income",
    income_source="Income source",
    fill_in_eitc_ctc="Fill-in EITC/CTC",
    mtr="Fill-in EITC/CTC marginal tax rate",
    children="Children",
    adults="Adults",
)

fig = px.line(
    df,
    "employment_income",
    "fill_in_eitc_ctc",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Earned income tax credit, 2022",
    color_discrete_map=COLOR_MAP,
)

fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
fig.show()

The EITC has marginal tax rates ranging from -45% to +21%, depending on the household structure and income.

In [6]:
fig = px.line(
    df,
    "employment_income",
    "mtr",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Fill-in EITC/CTC marginal tax rate, 2022",
    color_discrete_map=COLOR_MAP,
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat=".0%",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
fig.show()